In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as ss
import openpyxl

from generate_func import SkeletalSFactors

I did run!
I produced something!


# S-factor calculation

Energy data from DECDATA, not from Firestone.
Separated into IC-electrons, beta-minus-electrons and auger-electrons.
Using Hough-UF-model for the male skeleton first, redo and re-check calculations.

We do have the specific energy absorbtion fractions for the lumbar vertebraes in a tabular format.

27/7/2020

Better still, make it general so we can have multiple skeletal sites and nuclides, to better get a grasp of the (full) model.

Need to convert some values from the PDF, direct import in excel returns only text-strings. This can be done better

Finnished now, have to re-calculate everything but finally we have something usefull. Set up the new analysis and the leave-1-out and all of the other stuff...

5/8/2020

Continue writing a function of the class, one that takes the CF, mass and Cum and returns the absorbed dose - 

# Design of S-factors.

Need a class or object that can hold the data needed for easy get/set. The class should contain:

* Name of isotope
* A set of skeletal sites (should we here use a key-ed dictionary?)
* Nested in this should be the CFs and the trabular corrections

In [ ]:
# Snippet to test the class-constructing function:

saf_input = "utils/saf_values_male_redone.xlsx"
saf_input = "utils/saf_values_female_redone.xlsx"

from generate_func import create_skeletal_phantom

In [ ]:
from generate_func import SkeletalSFactors

#(energy_frame, cf_dict, tbv_fracs) = create_skeletal_phantom(
#    saf_file_path="utils/saf_values_male_redone.xlsx",
#    isotope_data_path="utils/decdataenergies.xlsx")

MaleFactor = SkeletalSFactors(
    isotopename = "177-Lu",
    phantomsex = "M",
    saf_file="utils/saf_values_male_redone.xlsx",
    isotope_file = "utils/decdataenergies.xlsx")

#(energy_frame, cf_dict, tbv_fracs) = create_skeletal_phantom(
#    saf_file_path="utils/saf_values_female_redone.xlsx",
#    isotope_data_path="utils/decdataenergies.xlsx")

#FemaleFactor = SkeletalSFactors(
#    isotopename = "177-Lu",
#    phantomsex = "F",
#    skeletalfactors = energy_frame,
#    icrp_cf_vals=cf_dict,
#    trabecular_corrections=tbv_fracs)

#print(MaleFactor.skeletalfactors["LumbarVertebrae"])
#print(FemaleFactor.skeletalfactors["LumbarVertebrae"])

In [ ]:
foobar = (MaleFactor.skeletalfactors)
foo = (foobar["LumbarVertebrae"])

print(foobar)

CF = "CF10"
cf_val = float((CF[2::]))/100.0
tbc_c = ((MaleFactor.trabecular_corrections)["LumbarVertebrae"][0])


volume = 0.01*cf_val*(1-tbc_c)
cum_act = 120*3600*1e6

E = (foo["EnergyPerDesintegration"][CF])
print((E*cum_act*1.60217662e-13)/volume)

In [2]:
TH_data_path = "../Betalutin/ErebusArticle/analysis/input/THData.xlsx"
lumbar_data_path = "../Betalutin/ErebusArticle/analysis/input/lumbardata.xlsx"

MaleFactor = SkeletalSFactors(
    isotopename = "177-Lu",
    phantomsex = "M",
    saf_file="utils/saf_values_male_redone.xlsx",
    isotope_file = "utils/decdataenergies.xlsx")
  
FemaleFactor = SkeletalSFactors(
    isotopename = "177-Lu",
    phantomsex = "F",
    saf_file="utils/saf_values_female_redone.xlsx",
    isotope_file = "utils/decdataenergies.xlsx")




In [4]:
input_frame = pd.read_excel(TH_data_path)

print(input_framerame)

                    Site  SiteID     Pnum         Mass          Bqs
0                OsCoxae     OCL      P19      159.021  6.66475e+12
1    Hip bone right mass     OCR      P19      143.655  7.18814e+12
2                L5 mass  lumbar      P19       21.801  2.16912e+12
3                L4 mass  lumbar      P19       27.041   2.2495e+12
4                L3 mass  lumbar      P19      31.4512  2.21709e+12
..                   ...     ...      ...          ...          ...
279                  NaN     NaN      NaN          NaN          NaN
280                  NaN     NaN      NaN          NaN          NaN
281                  NaN     NaN      NaN          NaN          NaN
282      LumbarVertebrae     P24  132.515  6.74127e+12            F
283      LumbarVertebrae     P25   112.63  1.19061e+13            M

[284 rows x 5 columns]


In [ ]:
foo = MaleFactor.skeletalfactors

print(MaleFactor.calculate_dose.__doc__)
#print((MaleFactor.skeletalfactors)["EnergyPerDesintegration"])

In [10]:
input_frame = pd.read_excel(TH_data_path, sheet_name="Input")



pbn = np.zeros(len(input_frame))
ad = np.zeros(len(input_frame))

for index_of_input in range(len(input_frame)):
    
    mass_of_marrow = (input_frame["Mass"][index_of_input])
    site_name = (input_frame["Site"][index_of_input])
    cum_act = (input_frame["Bqs"][index_of_input])
    p_sex = (input_frame["Sex"][index_of_input])
    p_num = (input_frame["Pnum"][index_of_input])

    #print(1e-9*cum_act/mass_of_marrow)

    for CF in ["CF100", "ICRP"]:
    
        if p_sex == "M":

            absorbed_dose = (MaleFactor.calculate_dose(mass=mass_of_marrow, 
                                      cumulative_activity=cum_act,
                                      cf_string=CF,
                                      site_name= site_name))
        else:
            absorbed_dose = (FemaleFactor.calculate_dose(mass=mass_of_marrow, 
                                      cumulative_activity=cum_act,
                                      cf_string=CF,
                                      site_name= site_name))

        ad[index_of_input] = absorbed_dose
    


print(ad)

[1.16142134 1.38662074 1.99182009 2.80979312 1.59273372 1.72719094
 2.50839709 2.95605656 1.95275541 2.03309161 2.35477757 3.13072726
 4.18795048 1.83172476 1.94772092 2.49318493 2.63778906 3.9905804
 3.35465871 2.02728717 2.48338459 3.49991847 3.79795862 1.69316564
 1.82089952 2.38895594 3.13170158 1.90341765 1.84584773 1.87368288
 2.81311763 3.87507177 0.88633537 1.1266956  3.09442676 2.63462814
 2.59960579 3.82289437 3.35453904 3.582423   3.28473899 4.2849943
 4.7350556  2.62269149]


In [20]:
# Populate and make a data frame for containing the multi-site-analysis:


patients = np.unique(input_frame["Pnum"])
sites = np.unique(input_frame["Site"])
cfs = [
    "CF100",
    "CF90",
    "CF80",
    "CF70",
    "CF60",
    "CF50",
    "CF40",
    "CF30",
    "CF20",
    "CF10",
    "ICRP",
]

output_frame = pd.DataFrame(columns=)



index_of_input = 0

while index_of_input < 12:
    
    mass_of_marrow = (input_frame["Mass"][index_of_input])
    site_name = (input_frame["Site"][index_of_input])
    cum_act = (input_frame["Bqs"][index_of_input])
    p_sex = (input_frame["Sex"][index_of_input])
    p_num = (input_frame["Pnum"][index_of_input])

    #print(1e-9*cum_act/mass_of_marrow)

    for CF in cfs:
    
        if p_sex == "M":

            absorbed_dose = (MaleFactor.calculate_dose(mass=mass_of_marrow, 
                                      cumulative_activity=cum_act,
                                      cf_string=CF,
                                      site_name= site_name))
        else:
            absorbed_dose = (FemaleFactor.calculate_dose(mass=mass_of_marrow, 
                                      cumulative_activity=cum_act,
                                      cf_string=CF,
                                      site_name= site_name))
        
    
        

1.023749152157168 OsCoxae CF100
1.0367632239827667 OsCoxae CF90
1.056856539496867 OsCoxae CF80
1.07984325795893 OsCoxae CF70
1.1058442831609951 OsCoxae CF60
1.1503894465649813 OsCoxae CF50
1.2160565748734886 OsCoxae CF40
1.3135817723530556 OsCoxae CF30
1.5156396372921583 OsCoxae CF20
2.1062908738207247 OsCoxae CF10
1.1614213366346982 OsCoxae ICRP
2.961576499054486 LumbarVertebrae CF100
3.0046316826876365 LumbarVertebrae CF90
3.0588187434055234 LumbarVertebrae CF80
3.130727263717218 LumbarVertebrae CF70
3.219621527513439 LumbarVertebrae CF60
3.336202394512194 LumbarVertebrae CF50
3.5167668180596436 LumbarVertebrae CF40
3.8071004817531406 LumbarVertebrae CF30
4.388834765497958 LumbarVertebrae CF20
6.09536980234831 LumbarVertebrae CF10
3.130727263717218 LumbarVertebrae ICRP
